In [28]:
import imp
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

In [29]:
problem = imp.load_source('', 'problem.py')
X_df, y_array = problem.get_train_data()

In [36]:
X_encoded = X_df
external_data = pd.read_csv('submissions/starting_kit/external_data.csv')
external_data = external_data[['Date', 'AirPort', 'Mean TemperatureC', 'MeanDew PointC', 
                               'Mean Humidity', 'Mean VisibilityKm', 'Mean Wind SpeedKm/h', 
                               'CloudCover']]
external_data = external_data.rename(
            columns={'Date': 'DateOfDeparture', 'AirPort': 'Arrival'})
X_encoded = pd.merge(
            X_encoded, external_data, how='left',
            left_on=['DateOfDeparture', 'Arrival'],
            right_on=['DateOfDeparture', 'Arrival'],
            sort=False)
X_encoded.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,Mean TemperatureC,MeanDew PointC,Mean Humidity,Mean VisibilityKm,Mean Wind SpeedKm/h,CloudCover
0,2012-06-19,ORD,DFW,12.875000,9.812647,29,21,63,16,29,5
1,2012-09-10,LAS,DEN,14.285714,9.466734,25,-6,14,16,15,3
2,2012-10-05,DEN,LAX,10.863636,9.035883,19,16,77,13,8,5
3,2011-10-09,ATL,ORD,11.480000,7.990202,19,10,58,16,6,1
4,2012-02-21,DEN,SFO,11.450000,9.517159,12,8,79,12,8,7


In [37]:
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Departure'], prefix='d'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Arrival'], prefix='a'))
X_encoded = X_encoded.drop('Departure', axis=1)
X_encoded = X_encoded.drop('Arrival', axis=1)

X_encoded['DateOfDeparture'] = pd.to_datetime(X_encoded['DateOfDeparture'])
#X_encoded['year'] = X_encoded['DateOfDeparture'].dt.year
#X_encoded['month'] = X_encoded['DateOfDeparture'].dt.month
X_encoded['weekday'] = X_encoded['DateOfDeparture'].dt.weekday
#X_encoded['n_days'] = X_encoded['DateOfDeparture'].apply(lambda date: (date - pd.to_datetime("1970-01-01")).days)

#X_encoded = X_encoded.join(pd.get_dummies(X_encoded['year'], prefix='y'))
#X_encoded = X_encoded.join(pd.get_dummies(X_encoded['month'], prefix='m'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['weekday'], prefix='wd'))

#X_encoded = X_encoded.drop(['year', 'month', 'weekday', 'n_days'], axis=1)
X_encoded = X_encoded.drop(['weekday'], axis=1)

X_encoded = X_encoded.drop('DateOfDeparture', axis=1)

X_encoded.head()

,WeeksToDeparture,std_wtd,Mean TemperatureC,MeanDew PointC,Mean Humidity,Mean VisibilityKm,Mean Wind SpeedKm/h,CloudCover,d_ATL,d_BOS,d_CLT,d_DEN,d_DFW,d_DTW,d_EWR,d_IAH,d_JFK,d_LAS,d_LAX,d_LGA,d_MCO,d_MIA,d_MSP,d_ORD,d_PHL,d_PHX,d_SEA,d_SFO,a_ATL,a_BOS,a_CLT,a_DEN,a_DFW,a_DTW,a_EWR,a_IAH,a_JFK,a_LAS,a_LAX,a_LGA,a_MCO,a_MIA,a_MSP,a_ORD,a_PHL,a_PHX,a_SEA,a_SFO,wd_0,wd_1,wd_2,wd_3,wd_4,wd_5,wd_6
0,12.875000,9.812647,29,21,63,16,29,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,14.285714,9.466734,25,-6,14,16,15,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,10.863636,9.035883,19,16,77,13,8,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,11.480000,7.990202,19,10,58,16,6,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
4,11.450000,9.517159,12,8,79,12,8,7,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0


In [26]:
from sklearn.preprocessing import StandardScaler

X_encoded.loc[:, 'WeeksToDeparture' : 'CloudCover'] = StandardScaler().fit_transform(
    X_encoded.loc[:, 'WeeksToDeparture' : 'CloudCover'])
X_encoded.head()


,WeeksToDeparture,std_wtd,Mean TemperatureC,MeanDew PointC,Mean Humidity,Mean VisibilityKm,Mean Wind SpeedKm/h,CloudCover,d_ATL,d_BOS,d_CLT,d_DEN,d_DFW,d_DTW,d_EWR,d_IAH,d_JFK,d_LAS,d_LAX,d_LGA,d_MCO,d_MIA,d_MSP,d_ORD,d_PHL,d_PHX,d_SEA,d_SFO,a_ATL,a_BOS,a_CLT,a_DEN,a_DFW,a_DTW,a_EWR,a_IAH,a_JFK,a_LAS,a_LAX,a_LGA,a_MCO,a_MIA,a_MSP,a_ORD,a_PHL,a_PHX,a_SEA,a_SFO,y_2011,y_2012,y_2013,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9,m_10,m_11,m_12,wd_0,wd_1,wd_2,wd_3,wd_4,wd_5,wd_6
0,0.512573,0.558487,1.577511,1.598672,0.041457,0.527353,2.360079,0.284865,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
1,1.018752,0.396807,1.147021,-1.239761,-2.892734,0.527353,0.153223,-0.550402,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
2,-0.209127,0.195426,0.501287,1.073036,0.879797,-0.654346,-0.950204,0.284865,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,0.012031,-0.293328,0.501287,0.442273,-0.257950,0.527353,-1.265469,-1.385670,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
4,0.001267,0.420375,-0.252070,0.232019,0.999560,-1.048246,-0.950204,1.120133,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [15]:
import pandas as pd
import os


class FeatureExtractor(object):
    def __init__(self):
        pass

    def fit(self, X_df, y_array):
        pass

    def transform(self, X_df):
        X_encoded = X_df
        path = os.path.dirname(__file__)
        data_weather = pd.read_csv(os.path.join(path, 'external_data.csv'))
        X_weather = data_weather[['Date', 'AirPort', 'Max TemperatureC']]
        X_weather = X_weather.rename(
            columns={'Date': 'DateOfDeparture', 'AirPort': 'Arrival'})
        X_encoded = pd.merge(
            X_encoded, X_weather, how='left',
            left_on=['DateOfDeparture', 'Arrival'],
            right_on=['DateOfDeparture', 'Arrival'],
            sort=False)

        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Departure'], prefix='d'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Arrival'], prefix='a'))
        X_encoded = X_encoded.drop('Departure', axis=1)
        X_encoded = X_encoded.drop('Arrival', axis=1)

        X_encoded['DateOfDeparture'] = pd.to_datetime(X_encoded['DateOfDeparture'])
        X_encoded['year'] = X_encoded['DateOfDeparture'].dt.year
        X_encoded['month'] = X_encoded['DateOfDeparture'].dt.month
        X_encoded['day'] = X_encoded['DateOfDeparture'].dt.day
        X_encoded['weekday'] = X_encoded['DateOfDeparture'].dt.weekday
        X_encoded['week'] = X_encoded['DateOfDeparture'].dt.week
        X_encoded['n_days'] = X_encoded['DateOfDeparture'].apply(lambda date: (date - pd.to_datetime("1970-01-01")).days)

        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['year'], prefix='y'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['month'], prefix='m'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['day'], prefix='d'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['weekday'], prefix='wd'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['week'], prefix='w'))

        X_encoded = X_encoded.drop(['year', 'month', 'day', 'weekday', 'week', 'n_days'], axis=1)

        X_encoded = X_encoded.drop('DateOfDeparture', axis=1)
        X_array = X_encoded.values
        return X_array